In [2]:
pip install fastapi uvicorn pyngrok langchain_openai openai langdetect deep-translator httpx==0.23.0 httpcore==0.15.0 python-jose


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━

In [3]:
from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel
from jose import JWTError, jwt
from typing import Optional, List, Dict
from datetime import datetime, timedelta
from langchain_openai import ChatOpenAI
from langchain.tools import tool, Tool
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType
import xmlrpc.client
import uvicorn
import threading
from pyngrok import ngrok
import traceback
from langdetect import detect
from deep_translator import GoogleTranslator
import pytz

# JWT Configuration
SECRET_KEY = "xxxxx"  # Replace with a strong secret key
ALGORITHM = "xxxx"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

# Dummy user database
fake_users_db = {
    
}

# Define Token and Login Models
class Token(BaseModel):
    access_token: str
    token_type: str

class LoginForm(BaseModel):
    username: str
    password: str

# Define Agent Query Input Model
class AgentQueryInput(BaseModel):
    query: str
    token: str

# FastAPI app initialization
app = FastAPI()

# Set API keys
ngrok.set_auth_token("xxx")
openai_api_key = ""

# Helper Function: Language Detection and Translation
def detect_and_translate(response: str, query: str) -> str:
    """
    Detects the language of the query and translates the response to match.

    Args:
        response (str): The system-generated response in English.
        query (str): The user's query to detect its language.

    Returns:
        str: The translated response in the same language as the query.
    """
    try:
        # Ensure the query has valid text
        if not query or not query.strip():
            print("Query is empty or invalid. Defaulting to English response.")
            return response  # Default to English if query is empty or invalid

        # Detect the language of the query
        query_language = detect(query)
        print(f"Detected query language: {query_language}")

        # Translate the response if the query language is not English
        if query_language != "en":
            translated_response = GoogleTranslator(source='en', target=query_language).translate(response)
            print(f"Original Response: {response}")
            print(f"Translated Response: {translated_response}")
            return translated_response

        return response  # Return original response if query language is English
    except Exception as e:
        print(f"Error in detect_and_translate: {str(e)}")
        return "Sorry, there was an error processing your request."


# Odoo Connector Class
class OdooConnector:
    def __init__(self, url, db, username, password):
        self.url = url
        self.db = db
        self.username = username
        self.password = password
        self.uid = None
        self.models = None
        self._authenticate()

    def _authenticate(self):
        common = xmlrpc.client.ServerProxy(f"{self.url}/xmlrpc/2/common")
        self.uid = common.authenticate(self.db, self.username, self.password, {})
        self.models = xmlrpc.client.ServerProxy(f"{self.url}/xmlrpc/2/object")

    def get_companies(self) -> List[Dict]:
        return self.models.execute_kw(
            self.db, self.uid, self.password,
            'res.partner', 'search_read',
            [[['is_company', '=', True]]],
            {'fields': ['name', 'id', 'country_id', 'email'], 'limit': 20}
        )

    def get_contacts(self) -> List[Dict]:
        return self.models.execute_kw(
            self.db, self.uid, self.password,
            'res.partner', 'search_read',
            [[['is_company', '=', False]]],
            {'fields': ['name', 'id', 'email', 'parent_id'], 'limit': 20}
        )

    def schedule_appointment(self, contact_id: int, appointment_datetime: str) -> str:
        try:
            berlin_tz = pytz.timezone("Europe/Berlin")
            utc_tz = pytz.UTC
            local_datetime = datetime.fromisoformat(appointment_datetime)
            berlin_datetime = berlin_tz.localize(local_datetime)
            utc_datetime = berlin_datetime.astimezone(utc_tz)

            event = {
                'name': f"Appointment with Contact {contact_id}",
                'start': utc_datetime.strftime('%Y-%m-%d %H:%M:%S'),
                'stop': utc_datetime.strftime('%Y-%m-%d %H:%M:%S'),
                'partner_ids': [(4, contact_id)],
            }

            event_id = self.models.execute_kw(
                self.db, self.uid, self.password,
                'calendar.event', 'create', [event]
            )
            return f"Appointment successfully scheduled with contact ID {contact_id} on {appointment_datetime} (Europe/Berlin time)."
        except Exception as e:
            return f"Error scheduling appointment: {str(e)}"

    def get_contact_id_by_name(self, name: str) -> Optional[int]:
        try:
            contacts = self.models.execute_kw(
                self.db, self.uid, self.password,
                'res.partner', 'search_read',
                [[['is_company', '=', False], ['name', '=', name]]],
                {'fields': ['id'], 'limit': 1}
            )
            if contacts:
                return contacts[0]['id']
            return None
        except Exception as e:
            return None

# Initialize Odoo
odoo = OdooConnector(
    url='xx',
    db='xx',
    username='xx',
    password='xx'
)

# Authentication Utilities
def authenticate_user(username: str, password: str):
    user = fake_users_db.get(username)
    if user and user["password"] == password:
        return user
    return None

def create_access_token(data: dict, expires_delta: Optional[timedelta] = None):
    to_encode = data.copy()
    if expires_delta:
        expire = datetime.utcnow() + expires_delta
    else:
        expire = datetime.utcnow() + timedelta(hours=2)
    to_encode.update({"exp": expire})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

def verify_token(token: str):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username = payload.get("sub")
        if username is None:
            raise HTTPException(status_code=401, detail="Invalid token")
        return username
    except JWTError:
        raise HTTPException(status_code=401, detail="Invalid token")

# Login Endpoint
@app.post("/login", response_model=Token)
def login(form_data: LoginForm):
    user = authenticate_user(form_data.username, form_data.password)
    if not user:
        raise HTTPException(status_code=401, detail="Invalid username or password")
    access_token = create_access_token(data={"sub": user["username"], "role": user["role"]})
    return {"access_token": access_token, "token_type": "bearer"}


# Tools for LangChain Agent
@tool
def get_companies_tool(query: str) -> str:
    """
    Tool to fetch a list of companies from Odoo.

    Args:
        query: The user's query.

    Returns:
        str: A stringified list of companies.
    """
    try:
        print(f"Received query: {query}")  # Log the incoming query

        # Fetch companies from Odoo
        companies = odoo.get_companies()
        if not companies:
            print("No companies found in Odoo.")
            response = "No companies found in the system."
        else:
            response = "\n".join([f"{company['name']} (Email: {company.get('email', 'N/A')})" for company in companies])

        # Detect language and translate response
        translated_response = detect_and_translate(response, query)
        print(f"Translated response: {translated_response}")
        return translated_response

    except Exception as e:
        print(f"Error in get_companies_tool: {str(e)}")
        return detect_and_translate(
            "I'm sorry, I couldn't retrieve the list of companies due to a technical issue. Please try again later.", query
        )


@tool
def get_contacts_tool(query: str) -> str:
    """
    Tool to fetch a list of individual contacts from Odoo.

    Returns:
        str: A stringified list of contacts.
    """
    try:
        print(f"Received query: {query}")  # Log the incoming query

        # Fetch contacts from Odoo
        contacts = odoo.get_contacts()
        if not contacts:
            print("No contacts found in Odoo.")
            response = "No contacts found in the system."
        else:
            response = "\n".join([f"{contact['name']} (Email: {contact.get('email', 'N/A')})" for contact in contacts])

        # Detect language and translate response
        translated_response = detect_and_translate(response, query)
        print(f"Translated response: {translated_response}")
        return translated_response

    except Exception as e:
        print(f"Error in get_contacts_tool: {str(e)}")
        return detect_and_translate(
            "Sorry, there was an error fetching the list of contacts.", query
        )


@tool
def schedule_appointment_tool(query: str) -> str:
    """
    Schedules an appointment with an employee or contact in Odoo by name or ID.

    Args:
        query (str): A string containing the contact name or ID and appointment datetime.
                     Example: "Schedule an appointment with Fahad on 2024-11-27 14:00"

    Returns:
        str: Confirmation message or error.
    """
    try:
        parts = query.split(" ")
        if "ID" in query:
            contact_id = int(parts[6])
            appointment_datetime = " ".join(parts[8:])
        else:
            contact_name = parts[4]
            appointment_datetime = " ".join(parts[6:])
            contact_id = odoo.get_contact_id_by_name(contact_name)
            if not contact_id:
                return detect_and_translate(
                    f"I'm sorry, but I can't schedule the appointment as I can't find the ID of {contact_name}.", query)

        result = odoo.schedule_appointment(contact_id, appointment_datetime)
        return detect_and_translate(result, query)
    except (IndexError, ValueError):
        return detect_and_translate("Invalid query format.", query)
    except Exception as e:
        return detect_and_translate(
            "Sorry, there was an error scheduling the appointment.", query
        )

@tool
def search_employee_tool(query: str) -> str:
    """
    Tool to search for a contact within a company.

    Args:
        query: A string containing the contact ID and company ID,
               separated by a comma.
               Example: "123,456" where 123 is the contact ID
               and 456 is the company ID.

    Returns:
        str: A message indicating whether the contact is associated with the company.
    """
    try:
        employee_name = " ".join(query.split(" ")[-1:])
        employee_id = odoo.get_contact_id_by_name(employee_name)

        if employee_id:
            response = f"Employee {employee_name} found with ID {employee_id}."
        else:
            response = f"Employee {employee_name} not found."

        return detect_and_translate(response, query)
    except Exception as e:
        print(f"Error in search_employee_tool: {str(e)}")
        return detect_and_translate(
            "Sorry, there was an error searching for the employee.", query
        )

# LangChain Agent Initialization
model = ChatOpenAI(model="gpt-4", api_key=openai_api_key)
tools = [get_companies_tool, get_contacts_tool, schedule_appointment_tool, search_employee_tool]
agent_executor = initialize_agent(tools, model, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

# Agent Query Endpoint
@app.post("/agent_query")
async def agent_query(input_data: AgentQueryInput):
    try:
        # Log the incoming query
        print(f"Received query: {input_data.query}")
        query = input_data.query
        print(f"Raw query received: {input_data.query}") # Ensure query is not empty or stripped

        if not query.strip():
            return {"response": "The query cannot be empty. Please provide a valid input."}

        # Decode and verify the token
        token = input_data.token
        jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])

        # Log the agent input
        agent_input = {"input": query}
        print(f"Agent input: {agent_input}")

        # Run the agent
        response = agent_executor.run(agent_input)
        print(f"Agent response before translation: {response}")

        # Translate the response
        translated_response = detect_and_translate(response, query)
        print(f"Translated response: {translated_response}")

        return {"response": translated_response}
    except jwt.ExpiredSignatureError:
        return {"error": "Token has expired"}
    except jwt.JWTError as e:
        return {"error": f"Invalid token: {str(e)}"}
    except Exception as e:
        traceback.print_exc()
        return {"error": f"Internal Server Error: {str(e)}"}



# Run the app with ngrok
def run():
    uvicorn.run(app, host="0.0.0.0", port=8001)

ngrok_tunnel = ngrok.connect(8001)
print(f"ngrok URL: {ngrok_tunnel.public_url}")

thread = threading.Thread(target=run)
thread.start()


<ipython-input-3-b5ecfe9fb040>:332: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent_executor = initialize_agent(tools, model, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)


ngrok URL: https://475e-104-155-196-141.ngrok-free.app


INFO:     Started server process [345]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)
